In [1]:
import os
import pandas as pd
import numpy as np
import json
import difflib as dl
import funcTest as ft

In [72]:
os.chdir('F:\\TFG\\datasets\\nature-dataset')

# loading the events data
events={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open('./events/events_%s.json' %nation) as json_data:
        events[nation] = json.load(json_data)
        
# loading the match data
matches={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open('./matches/matches_%s.json' %nation) as json_data:
        matches[nation] = json.load(json_data)

# loading the players data
players={}
with open('./players.json') as json_data:
    players = json.load(json_data)
    
# loading the teams data
teams={}
with open('./teams.json',encoding='utf-8') as json_data:
    teams = json.load(json_data)

# loading the competitions data
competitions={}
with open('./competitions.json') as json_data:
    competitions = json.load(json_data)
    
# loading the playerank data
playerank={}
with open('./playerank.json') as json_data:
    playerank = json.load(json_data)
    
# # loading the referees data
# referees={}
# with open('./referees.json') as json_data:
#     referees = json.load(json_data)

# loading the coaches data
coaches={}
with open('./coaches.json') as json_data:
    coaches = json.load(json_data)

<h3>Vamos a concatenar todos los partidos de las 5 ligas</h3>

In [73]:
partidos = []

paises = ['Italy','England','Germany','France','Spain']

for pais in paises:
    for partido in matches[pais]:
        partidos.append(partido)
        
partidos = np.array(partidos,dtype=dict)
partidos.shape

(1826,)

In [74]:
partidos

array([{'status': 'Played', 'roundId': 4406278, 'gameweek': 38, 'teamsData': {'3162': {'scoreET': 0, 'coachId': 251025, 'side': 'home', 'teamId': 3162, 'score': 2, 'scoreP': 0, 'hasFormation': 1, 'formation': {'bench': [{'playerId': 4792, 'ownGoals': '2', 'redCards': '93', 'goals': '0', 'yellowCards': '0'}, {'playerId': 20972, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 256479, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 241675, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 260250, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 69417, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 20460, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 101635, 'ownGoals': '0', 'redCards': '0', 'goals': 'null', 'yellowCards': '0'}, {'playerId': 20575, 'ownGoals': '0', 'redCa

<h3>Crear tabla de partidos y filtrar atributos</h3>

<h4>Primero, agrupamos los datos que deseamos para los datasets</h4>

<b>Dataset 1</b>: será un dataset más simple simplemente con resultados  y IDs de equipos, competiciones, temporadas y nº de jornada

    - ID Partido
    - ID Competicion
    - ID Temporada
    - Jornada (roundID)
    - IDs partidos para cada equipo -> se calcula
    - Fecha (dateutc) -> Formato YYYY-MM-DD hh:mm:ss
    - ID Equipos
    - Nombre Equipos
    - Goles Equipos
    - ID Ganador o 0 si empate
    - Formaciones Equipos (si tienen 1 sino 0)

In [75]:
pd.DataFrame(partidos)
s = pd.Series(partidos[0])
s.index

Index(['status', 'roundId', 'gameweek', 'teamsData', 'seasonId', 'dateutc',
       'winner', 'venue', 'wyId', 'label', 'date', 'referees', 'duration',
       'competitionId'],
      dtype='object')

In [76]:
col = ['a',
       'b']

In [77]:
match_serie = [pd.Series(p) for p in partidos]
# print(match_serie)
rawDB = pd.DataFrame(match_serie, columns=(match_serie[0]).index)

columnNamesDB = ['wyId','competitionId','seasonId','roundId','dateutc','winner'] # wyId = ID del partido en el DB
rawDB = rawDB[columnNamesDB]

In [78]:
rawDB

,wyId,competitionId,seasonId,roundId,dateutc,winner
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163
...,...,...,...,...,...,...
1821,2565552,795,181144,4406122,2017-08-19 20:15:00,0
1822,2565551,795,181144,4406122,2017-08-19 18:15:00,0
1823,2565549,795,181144,4406122,2017-08-19 16:15:00,687
1824,2565556,795,181144,4406122,2017-08-18 20:15:00,674


<b>Añadimos ahora los datos de cada equipo:</b>

Para añadirlos hay que tener en cuenta que los datos de cada equipo se encuentran como un objeto JSON dentro del JSON del partido, es decir dentro del atributo <b>teamsData</b>

Dentro del atributo teamsData encontramos dos JSON, uno para cada equipo. El índice de cada JSON es el Id del equipo

In [79]:
teamsData = [list(m['teamsData'].values()) for m in match_serie] # partido x equipo

homeT = []
awayT = []

for m in teamsData:
#     print(m[0]['teamId'])
    if m[0]['side'] == 'home':
        homeT.append(pd.Series(m[0]))
        awayT.append(pd.Series(m[1]))
    else:
        homeT.append(pd.Series(m[1]))
        awayT.append(pd.Series(m[0]))
           
# print(set([x['teamId'] for x in homeT]))

# debemos cambiar el nombre de las columnas para que no se llamen igual las Home que las Away

colH = [str(x) + '_home' for x in homeT[0].index]
colA = [str(x) + '_away' for x in awayT[0].index]

hData = pd.DataFrame(homeT, columns=homeT[0].index)
aData = pd.DataFrame(awayT, columns=awayT[0].index)
hData.columns = colH
aData.columns = colA

# Eliminamos los atributos que no nos interesan


# Añadimos el nombre del equipo -> hay que buscarlo en el JSON de equipos

# homeNames = [teams['wyId'==hT['teamId']]['name'] for hT in homeT]
# awayNames = [teams['wyId'==aT['teamId']]['name'] for aT in awayT]
# print(homeNames[500])

# hData = hData.insert(len(hData.columns),)

assert len(hData) == len(aData) == len(rawDB)

In [80]:
hData[:3]

,scoreET_home,coachId_home,side_home,teamId_home,score_home,scoreP_home,hasFormation_home,formation_home,scoreHT_home
0,0,251025,home,3162,2,0,1,"{'bench': [{'playerId': 4792, 'ownGoals': '2',...",2
1,0,20603,home,3315,0,0,1,"{'bench': [{'playerId': 423151, 'ownGoals': '0...",0
2,0,251044,home,3173,1,0,1,"{'bench': [{'playerId': 402898, 'ownGoals': '0...",0


In [81]:
rawDB = pd.concat([rawDB,hData,aData],axis=1)

In [83]:
# rawDB.to_csv('F:\\TFG\\datasets\\data_train\\rawDB.csv',index=False)

Ahora deberemos seleccionar y arreglar los datos que queremos

In [84]:
rawDB.columns

Index(['wyId', 'competitionId', 'seasonId', 'roundId', 'dateutc', 'winner',
       'scoreET_home', 'coachId_home', 'side_home', 'teamId_home',
       'score_home', 'scoreP_home', 'hasFormation_home', 'formation_home',
       'scoreHT_home', 'scoreET_away', 'coachId_away', 'side_away',
       'teamId_away', 'score_away', 'scoreP_away', 'hasFormation_away',
       'formation_away', 'scoreHT_away'],
      dtype='object')

In [85]:
rawDB = rawDB[[
    'wyId', 'competitionId', 'seasonId', 'roundId', 'dateutc', 'winner',
    'teamId_home','score_home','scoreHT_home','teamId_away', 'score_away',
    'scoreHT_away'
     ]]

assert len(rawDB) == 1826 == len(set(rawDB['wyId']))

rawDB.loc[360]

wyId                         2575970
competitionId                    524
seasonId                      181248
roundId                      4406278
dateutc          2017-08-27 18:45:00
winner                          3162
teamId_home                     3165
score_home                         1
scoreHT_home                       1
teamId_away                     3162
score_away                         2
scoreHT_away                       1
Name: 360, dtype: object

In [86]:
local = []
visit = []

for row in rawDB['teamId_home']:
    for t in teams:
        local.append(t['name']) if t['wyId']==row else ''
        
for row in rawDB['teamId_away']:
    for t in teams:
        visit.append(t['name']) if t['wyId']==row else ''

In [87]:
assert len(local) == len(visit) == 1826

In [88]:
# Sacar nombre de los equipos

for t in  teams:
    print(t['name']) if t['wyId']==3161 else ''
        

Internazionale


<h3>Datasets que vamos a hacer</h3>

    - Cuotas con Victoria local (1) o otro resultado (0) -> cuotas_2
    - Cuotas con Victoria local (-1) empate (0) o victoria visitante (1) -> cuotas_3
    - Cuotas binaria + ultimos N partidos -> cuotas_racha_N
    - Cuotas binaria + ultimos N partidos + goles N partidos -> cuotas_racha_goles_N
    - Otras: 
        - teniendo en cuenta clasificaciones pasadas
        - teniendo en cuenta formación
        - teniendo en cuenta 

<b>Añadimos al dataset global las cuotas</b>

Habrá que identificar cada partido en ambas bases de datos

In [45]:
os.chdir('F:\\TFG\\datasets\\football-data\\')
dir_footdata = os.getcwd()

In [46]:
      
#LALIGA
data_fd = {}
data_fd['Esp'] = pd.read_csv(dir_footdata +'\\SP1_'+ '1718' +'.csv')
       
#PREMIER
data_fd['Ing'] = pd.read_csv(dir_footdata +'\\E0_'+ '1718' +'.csv')

#BUNDESLIGA
data_fd['Ale'] = pd.read_csv(dir_footdata +'\\D1_'+ '1718' +'.csv')

#LIGUE 1
data_fd['Fra'] = pd.read_csv(dir_footdata +'\\F1_'+ '1718' +'.csv')

#SERIE A
data_fd['Ita'] = pd.read_csv(dir_footdata +'\\I1_'+ '1718' +'.csv')


In [47]:
data_fd['Esp']

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,SP1,18/08/17,Leganes,Alaves,1,0,H,1,0,H,...,1.46,18,-0.50,2.07,2.03,1.90,1.86,1.98,3.35,4.63
1,SP1,18/08/17,Valencia,Las Palmas,1,0,H,1,0,H,...,2.27,16,-0.75,2.05,1.97,1.96,1.91,1.78,4.24,4.43
2,SP1,19/08/17,Celta,Sociedad,2,3,A,1,1,D,...,1.84,18,-0.25,2.08,2.05,1.87,1.83,2.12,3.53,3.74
3,SP1,19/08/17,Girona,Ath Madrid,2,2,D,2,0,H,...,1.74,16,1.25,1.77,1.75,2.25,2.16,6.93,3.83,1.63
4,SP1,19/08/17,Sevilla,Espanol,1,1,D,1,1,D,...,2.09,16,-1.00,2.12,2.06,1.86,1.82,1.64,4.18,5.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,SP1,19/05/18,Villarreal,Real Madrid,2,2,D,0,2,A,...,3.79,19,0.25,2.05,1.98,1.93,1.87,4.74,5.05,1.62
376,SP1,20/05/18,Ath Bilbao,Espanol,0,1,A,0,1,A,...,2.06,17,-0.50,2.06,2.02,1.88,1.85,1.95,3.77,4.05
377,SP1,20/05/18,Ath Madrid,Eibar,2,2,D,1,1,D,...,1.98,19,-1.00,2.09,2.04,1.87,1.82,1.47,4.25,8.80
378,SP1,20/05/18,Barcelona,Sociedad,1,0,H,0,0,D,...,5.04,19,-2.00,1.94,1.89,2.03,1.97,1.31,6.40,8.60


In [48]:
data_fd['Esp'][data_fd['Esp'].columns[:25]]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,SP1,18/08/17,Leganes,Alaves,1,0,H,1,0,H,...,18,4,2,0,1,0,0,2.05,3.20,4.10
1,SP1,18/08/17,Valencia,Las Palmas,1,0,H,1,0,H,...,13,5,2,3,3,0,1,1.75,3.80,4.50
2,SP1,19/08/17,Celta,Sociedad,2,3,A,1,1,D,...,11,5,4,3,1,0,0,2.38,3.25,3.20
3,SP1,19/08/17,Girona,Ath Madrid,2,2,D,2,0,H,...,15,6,0,2,4,0,1,8.00,4.33,1.45
4,SP1,19/08/17,Sevilla,Espanol,1,1,D,1,1,D,...,12,7,3,2,4,1,0,1.62,4.00,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,SP1,19/05/18,Villarreal,Real Madrid,2,2,D,0,2,A,...,10,4,5,4,3,0,0,3.10,4.10,2.10
376,SP1,20/05/18,Ath Bilbao,Espanol,0,1,A,0,1,A,...,11,7,7,0,1,0,0,2.04,3.50,3.60
377,SP1,20/05/18,Ath Madrid,Eibar,2,2,D,1,1,D,...,14,2,2,6,2,1,0,1.61,3.80,6.00
378,SP1,20/05/18,Barcelona,Sociedad,1,0,H,0,0,D,...,13,3,8,3,2,0,0,1.22,7.00,11.00


In [49]:
fdataDB = pd.concat([data_fd['Esp'],data_fd['Ing'],data_fd['Ale'],data_fd['Fra'],data_fd['Ita']],ignore_index=True)
fdataDB[:4]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Referee
0,SP1,18/08/17,Leganes,Alaves,1,0,H,1,0,H,...,18,-0.50,2.07,2.03,1.90,1.86,1.98,3.35,4.63,NaN
1,SP1,18/08/17,Valencia,Las Palmas,1,0,H,1,0,H,...,16,-0.75,2.05,1.97,1.96,1.91,1.78,4.24,4.43,NaN
2,SP1,19/08/17,Celta,Sociedad,2,3,A,1,1,D,...,18,-0.25,2.08,2.05,1.87,1.83,2.12,3.53,3.74,NaN
3,SP1,19/08/17,Girona,Ath Madrid,2,2,D,2,0,H,...,16,1.25,1.77,1.75,2.25,2.16,6.93,3.83,1.63,NaN


In [50]:
fdataDB.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY',
       'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD',
       'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5',
       'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA',
       'PSCH', 'PSCD', 'PSCA', 'Referee'],
      dtype='object')

In [51]:
fdataDB = fdataDB[['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG', 'FTR','HS',
                   'AS','HST','AST','HC','AC','HF','AF','B365H', 'B365D', 'B365A']]
fdataDB.loc[[0,100,400,700,1000,1400,1700]]

assert len(fdataDB) == len(rawDB)

Hay que identificar los equipos para identificar los partidos

In [52]:
fdTeams = set(fdataDB['HomeTeam'])

# equipos nature dataset

naTeams = [t['name'] for t in teams if t['type']=='club']

assert len(fdTeams)==len(naTeams)==98

In [53]:
iguales = [t for t in fdTeams if t in naTeams]
len(iguales) # tenemos que 58 clubs se llaman igual en ambos datasets

desiguales = [t for t in naTeams if t not in iguales]

In [54]:
len(iguales)

58

In [55]:
encontrados = []
no_encontrados = []

for des in desiguales:
    find = dl.get_close_matches(des,fdTeams)
    if len(find)>0:
        encontrados.append((des,find[0]))
    else:
        no_encontrados.append(des)
        

In [56]:
len(encontrados)
encontrados # (na, fd)

[('Newcastle United', 'Newcastle'),
 ('Espanyol', 'Espanol'),
 ("Borussia M'gladbach", "M'gladbach"),
 ('Huddersfield Town', 'Huddersfield'),
 ('Atl\\u00e9tico Madrid', 'Ath Madrid'),
 ('Legan\\u00e9s', 'Leganes'),
 ('Swansea City', 'Swansea'),
 ('Olympique Marseille', 'Marseille'),
 ('AFC Bournemouth', 'Bournemouth'),
 ('Bayer Leverkusen', 'Leverkusen'),
 ('Borussia Dortmund', 'Dortmund'),
 ('Bayern M\\u00fcnchen', 'Bayern Munich'),
 ('Hannover 96', 'Hannover'),
 ('Hertha BSC', 'Hertha'),
 ('Hamburger SV', 'Hamburg'),
 ('Leicester City', 'Leicester'),
 ('West Ham United', 'West Ham'),
 ('Stoke City', 'Stoke'),
 ('West Bromwich Albion', 'West Brom'),
 ('Manchester City', 'Man City'),
 ('Tottenham Hotspur', 'Tottenham'),
 ('Mainz 05', 'Mainz'),
 ('Eintracht Frankfurt', 'Ein Frankfurt'),
 ('Manchester United', 'Man United'),
 ('Amiens SC', 'Amiens'),
 ('Hellas Verona', 'Verona'),
 ('Real Betis', 'Betis'),
 ('Real Sociedad', 'Sociedad')]

In [57]:
emparejadosFD = iguales
emparejadosNA = iguales

for (x,y) in encontrados:
    emparejadosFD.append(y)
    emparejadosNA.append(x)
    
desigualesFD = [t for t in fdTeams if t not in emparejadosFD]
desigualesNA = [t for t in naTeams if t not in emparejadosNA]

In [58]:
assert len(desigualesFD) == 12 == len(desigualesNA)

desigualesNA

['Celta de Vigo',
 'Deportivo Alav\\u00e9s',
 'Athletic Club',
 'Olympique Lyonnais',
 'PSG',
 'SPAL',
 'Brighton & Hove Albion',
 'Internazionale',
 'Saint-\\u00c9tienne',
 'K\\u00f6ln',
 'Deportivo La Coru\\u00f1a',
 'M\\u00e1laga']

In [59]:
desigualesFD

['Inter',
 'Spal',
 'Lyon',
 'Brighton',
 'Malaga',
 'FC Koln',
 'Alaves',
 'Paris SG',
 'Celta',
 'St Etienne',
 'Ath Bilbao',
 'La Coruna']

In [60]:
encontrados

[('Newcastle United', 'Newcastle'),
 ('Espanyol', 'Espanol'),
 ("Borussia M'gladbach", "M'gladbach"),
 ('Huddersfield Town', 'Huddersfield'),
 ('Atl\\u00e9tico Madrid', 'Ath Madrid'),
 ('Legan\\u00e9s', 'Leganes'),
 ('Swansea City', 'Swansea'),
 ('Olympique Marseille', 'Marseille'),
 ('AFC Bournemouth', 'Bournemouth'),
 ('Bayer Leverkusen', 'Leverkusen'),
 ('Borussia Dortmund', 'Dortmund'),
 ('Bayern M\\u00fcnchen', 'Bayern Munich'),
 ('Hannover 96', 'Hannover'),
 ('Hertha BSC', 'Hertha'),
 ('Hamburger SV', 'Hamburg'),
 ('Leicester City', 'Leicester'),
 ('West Ham United', 'West Ham'),
 ('Stoke City', 'Stoke'),
 ('West Bromwich Albion', 'West Brom'),
 ('Manchester City', 'Man City'),
 ('Tottenham Hotspur', 'Tottenham'),
 ('Mainz 05', 'Mainz'),
 ('Eintracht Frankfurt', 'Ein Frankfurt'),
 ('Manchester United', 'Man United'),
 ('Amiens SC', 'Amiens'),
 ('Hellas Verona', 'Verona'),
 ('Real Betis', 'Betis'),
 ('Real Sociedad', 'Sociedad')]

In [65]:
aux_equipos = ['Celta','Alaves','Ath Bilbao','Lyon','Paris SG','Spal','Brighton','Inter',
               'St Etienne','FC Koln','La Coruna','Malaga']

equipos = {} # { nature_data : football_data } }

for t in iguales:
    equipos[t] = t

for x,y in encontrados:
    equipos[x] = y
    
for x,y in zip(desigualesNA,aux_equipos):
    equipos[x] = y
    
# bug: los equipos 'encontrados' salen dupplicados?
for _,y in encontrados:
    if y in equipos.keys():
        del(equipos[y]) 
        
assert len(equipos) == 98

In [66]:
# comprobamos que todos los elementos coinciden

x = 0

for e in equipos.keys():
    x = x + naTeams.count(e)

assert x == len(naTeams) == len(equipos.keys())

In [67]:
equipos['Deportivo La Coru\\u00f1a']

'La Coruna'

Ahora debemos encontrar asociar los equipos de football_data con los de nature_data

In [68]:
nat_ID_Names = {}

for t in teams:
    if t['type'] == 'club':
        nat_ID_Names[t['wyId']] = t['name']
        
assert len(nat_ID_Names.keys()) == 98 == len(equipos.keys())

aux = equipos
equipos = {}

equipos['equipos'] = aux
equipos['nat_ID_Names'] = nat_ID_Names

with open('F:\\TFG\\datasets\\data_train\\equipos.json','w') as js:
    json.dump(equipos,js)

In [69]:
rawDB

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,scoreHT_away
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,3,1
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,1,1
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,0,0
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,3165,1,0,3219,0,0
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,3163,1,1,3166,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2565552,795,181144,4406122,2017-08-19 20:15:00,0,680,1,1,691,1,1
1822,2565551,795,181144,4406122,2017-08-19 18:15:00,0,756,2,2,679,2,0
1823,2565549,795,181144,4406122,2017-08-19 16:15:00,687,692,2,1,687,3,1
1824,2565556,795,181144,4406122,2017-08-18 20:15:00,674,674,1,1,714,0,0


<b> Vamos a añadir ya las cuotas al dataset de Nature que hemos creado anteriormente </b>

Para ello debemos iterar sobre el dataframe de Nature y buscar en el dataset de Football-Data que partido coincide con esos rivales.

(Hay que tener en cuenta que si tuviesemos más de temporadas se repetiria un mismo enfrentamiento entre dos equipos distintos)

In [18]:
rawDB[:3]

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,scoreHT_away
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,3,1
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,1,1
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,0,0


In [58]:
fdataDB[:3]

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,B365H,B365D,B365A
0,SP1,18/08/17,Leganes,Alaves,1,0,H,16,6,9,3,4,2,14,18,2.05,3.20,4.1
1,SP1,18/08/17,Valencia,Las Palmas,1,0,H,22,5,6,4,5,2,25,13,1.75,3.80,4.5
2,SP1,19/08/17,Celta,Sociedad,2,3,A,16,13,5,6,5,4,12,11,2.38,3.25,3.2


In [59]:
len(rawDB)

1826

In [66]:
# Tarda mucho en ejecutarse

data_cuotas = ['HS','AS','HST','AST','HC','AC','HF','AF','B365H', 'B365D', 'B365A']

cuotaNatDB = rawDB
cuotasFeat = {}

for dc in data_cuotas:
    cuotasFeat[dc] = np.empty(shape=(len(rawDB)),dtype=object)
    
index = 0

for h,a in zip(rawDB['teamId_home'],rawDB['teamId_away']):
    for i in range(len(rawDB)):
        p = fdataDB.loc[i]
        h_name = equipos[nat_ID_Names[h]]
        a_name = equipos[nat_ID_Names[a]]
        if p['HomeTeam']==h_name and p['AwayTeam']==a_name:
            for dc in data_cuotas:
                cuotasFeat[dc].put(index,p[dc])
            index += 1
            break

for dc in data_cuotas:
    assert len(cuotasFeat[dc]) == len(rawDB)
    cuotaNatDB[dc] = cuotasFeat[dc]

# # Tarda mucho en ejecutarse

# bet_h_nat = []
# bet_a_nat = []

# for h,a in zip(rawDB['teamId_home'],rawDB['teamId_away']):
#     for i in range(1826):
#         p = fdataDB.loc[i]
#         h_name = equipos[nat_ID_Names[h]]
#         a_name = equipos[nat_ID_Names[a]]
#         if p['HomeTeam']==h_name and p['AwayTeam']==a_name:
#             bet_h_nat.append(p['B365H'])
#             bet_a_nat.append(p['B365A'])
#             break

# assert len(bet_h_nat) == len(bet_a_nat) == 1826

# cuotaNatDB = rawDB
# cuotaNatDB['B365H'] = bet_h_nat
# cuotaNatDB['B365A'] = bet_a_nat

In [52]:
# EXPORTAMOS CSV
# cuotaNatDB.to_csv('F:\\TFG\\datasets\\data_train\\cuotaNatDB.csv',index=False)
# cuotaNatDB.to_csv('F:\\TFG\\datasets\\data_train\\Nature_wStatsDB_v1.csv',index=False)


# IMPORTAMOS CSV
cuotaNatDB = pd.read_csv('F:\\TFG\\datasets\\data_train\\cuotaNatDB.csv')
statsNatDB_v1 = pd.read_csv('F:\\TFG\\datasets\\data_train\\Nature_wStatsDB_v1.csv')

In [53]:
statsNatDB_v1

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,AS,HST,AST,HC,AC,HF,AF,B365H,B365D,B365A
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,...,15,4,6,10,4,12,13,2.40,3.50,2.90
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,...,15,3,6,5,9,14,8,4.00,3.80,1.85
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,...,12,4,4,5,11,17,14,3.10,3.60,2.25
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,3165,1,0,3219,...,12,5,1,2,4,13,9,1.53,4.50,5.75
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,3163,1,1,3166,...,8,5,5,10,2,9,11,1.53,4.33,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2565552,795,181144,4406122,2017-08-19 20:15:00,0,680,1,1,691,...,9,4,6,7,3,14,12,1.62,4.00,5.50
1822,2565551,795,181144,4406122,2017-08-19 18:15:00,0,756,2,2,679,...,9,6,3,6,0,15,15,8.00,4.33,1.45
1823,2565549,795,181144,4406122,2017-08-19 16:15:00,687,692,2,1,687,...,13,5,6,5,4,12,11,2.38,3.25,3.20
1824,2565556,795,181144,4406122,2017-08-18 20:15:00,674,674,1,1,714,...,5,6,4,5,2,25,13,1.75,3.80,4.50


Creamos la base de datos para que la puede usar el Modelo

- -1 gana local
- 0 empate
- 1 gana visitante

In [40]:
cuotaNatDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv')
cuotaNatDB

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,scoreHT_away,B365H,B365A,tendencia_H,tendencia_A
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,3,1,2.40,2.90,2.2,1.8
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,1,1,4.00,1.85,2.4,2.6
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,0,0,3.10,2.25,0.8,2.2
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,3165,1,0,3219,0,0,1.53,5.75,1.4,1.4
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,3163,1,1,3166,0,0,1.53,6.00,0.8,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2565552,795,181144,4406122,2017-08-19 20:15:00,0,680,1,1,691,1,1,1.62,5.50,NaN,NaN
1822,2565551,795,181144,4406122,2017-08-19 18:15:00,0,756,2,2,679,2,0,8.00,1.45,NaN,NaN
1823,2565549,795,181144,4406122,2017-08-19 16:15:00,687,692,2,1,687,3,1,2.38,3.20,NaN,NaN
1824,2565556,795,181144,4406122,2017-08-18 20:15:00,674,674,1,1,714,0,0,1.75,4.50,NaN,NaN


In [79]:
data_cuotas

['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'B365H', 'B365D', 'B365A']

In [78]:
cuotasDB = cuotaNatDB[data_cuotas]

KeyError: "['AS', 'HC', 'HST', 'HS', 'HF', 'B365D', 'AC', 'AF', 'AST'] not in index"

In [75]:
def calcRES(db):
    res = []
    for h,a in zip(db['score_home'],db['score_away']):
        if h>a:
            res.append(-1)
        elif a>h:
            res.append(1)
        else:
            res.append(0) 
    
    assert len(res) == len(db)
    
    return res

In [76]:
# LA EXPORTAMOS
# cuotasDB.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB.csv',index=False)
# cuotaNatDB.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv',index=False)

# LO IMPORTAMOS
cuotaNatDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv')
cuotasDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB.csv')
cuotasDB[:5]

,B365H,B365A,res
0,2.40,2.90,1
1,4.00,1.85,1
2,3.10,2.25,-1
3,1.53,5.75,-1
4,1.53,6.00,-1


<b> Para el siguiente Dataset vamos a registrar la racha (resultados y goles) de los ultimos N partidos de un equipos </b>

In [41]:
cuotaNatDB.loc[cuotaNatDB['competitionId'] == 524][:5]

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,scoreHT_away,B365H,B365A
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,3,1,2.40,2.90
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,1,1,4.00,1.85
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,0,0,3.10,2.25
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,3165,1,0,3219,0,0,1.53,5.75
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,3163,1,1,3166,0,0,1.53,6.00


In [88]:
cuotaNatDB

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,score_away,scoreHT_away,B365H,B365A,tendencia_H,tendencia_A
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,3,1,2.40,2.90,2.2,1.8
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,1,1,4.00,1.85,2.4,2.6
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,0,0,3.10,2.25,0.8,2.2
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,3165,1,0,3219,0,0,1.53,5.75,1.4,1.4
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,3163,1,1,3166,0,0,1.53,6.00,0.8,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2565552,795,181144,4406122,2017-08-19 20:15:00,0,680,1,1,691,1,1,1.62,5.50,NaN,NaN
1822,2565551,795,181144,4406122,2017-08-19 18:15:00,0,756,2,2,679,2,0,8.00,1.45,NaN,NaN
1823,2565549,795,181144,4406122,2017-08-19 16:15:00,687,692,2,1,687,3,1,2.38,3.20,NaN,NaN
1824,2565556,795,181144,4406122,2017-08-18 20:15:00,674,674,1,1,714,0,0,1.75,4.50,NaN,NaN


In [67]:
def upt_features(feat,pts,hs_,goals,hst_,hc_,hf_,i):
    feat['points'] += pts
    feat['goals'] += goals
    feat['shots'] += hs_
    feat['on_target'] += hst_
    feat['corners'] += hc_
    feat['fouls'] += hf_
    return i+1
    
def descartar(feat,i,n):
    print(i, feat['goals'])
    if i < n:
        print('Partidos nulos')
        feat['points'] = np.NaN
        feat['goals'] = np.NaN
        feat['shots'] = np.NaN
        feat['on_target'] = np.NaN
        feat['corners'] = np.NaN
        feat['fouls'] = np.NaN    
    
def crear_tendencias(n=5,solo_local=True):
    
    features = {'ptsH': [], 'ptsA' : [], 'goalsH' : [], 'goalsA' : [],
                'HS' : [], 'AS' : [], 'HST' : [], 'AST' : [], 'HC' : [], 'AC' : [], 'HF' : [], 'AF' : []}
    id_comp = list(set(statsNatDB_v1['competitionId']))
    aux_df = statsNatDB_v1

    for comp in id_comp:
        rows = statsNatDB_v1.index[statsNatDB_v1['competitionId']==comp]
        df_comp = aux_df[rows[0]:rows[-1]+1]


        for index in range(rows[0],rows[-1]+1):
    #         print(index)
            ht = df_comp['teamId_home'][index]
            at = df_comp['teamId_away'][index]
            len_H = 0
            len_A = 0
            
            featH = {'points' : 0, 'goals' : 0, 'shots' : 0, 'on_target' : 0, 'corners' : 0, 'fouls' : 0}
            featA = {'points' : 0, 'goals' : 0, 'shots' : 0, 'on_target' : 0, 'corners' : 0, 'fouls' : 0}

            for i in range(index+1,rows[-1]+1):
                p = df_comp.loc[i]  
                h_aux = p['teamId_home']
                if ht == h_aux:
                    if p['score_home'] == p['score_away']:
                        len_H = upt_features(featH,1,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_H)
                    elif p['score_home'] > p['score_away']:
                        len_H = upt_features(featH,3,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_H)
                    else:
                        len_H = upt_features(featH,0,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_H)
                elif ht == p['teamId_away'] and solo_local:
                    if p['score_home'] == p['score_away']:
                        len_H = upt_features(featH,1,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_H)
                    elif p['score_home'] > p['score_away']:
                        len_H = upt_features(featH,0,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_H)
                    else:
                        len_H = upt_features(featH,3,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_H)

                if len_H == n:
                    break
                
            descartar(featH,len_H,n)

            for i in range(index+1,rows[-1]+1):
                p = df_comp.loc[i]
                a_aux = p['teamId_home']
                if at == a_aux and solo_local:
                    if p['score_home'] == p['score_away']:
                        len_A = upt_features(featA,1,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_A)
                    elif p['score_home'] > p['score_away']:
                        len_A = upt_features(featA,3,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_A)
                    else:
                        len_A = upt_features(featA,0,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_A)
                elif at == p['teamId_away']:
                    if p['score_home'] == p['score_away']:
                        len_A = upt_features(featA,1,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_A)
                    elif p['score_home'] > p['score_away']:
                        len_A = upt_features(featA,0,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_A)
                    else:
                        len_A = upt_features(featA,3,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_A)

                if len_A == n:
                    break

            descartar(featA,len_A,n)
        
            print()
            
            features['ptsH'].append(featH['points'])
            features['goalsH'].append(featH['goals'])
            features['HS'].append(featH['shots'])
            features['HST'].append(featH['on_target'])
            features['HC'].append(featH['corners'])
            features['HF'].append(featH['fouls'])

            features['ptsA'].append(featA['points'])
            features['goalsA'].append(featA['goals'])
            features['AS'].append(featA['shots'])
            features['AST'].append(featA['on_target'])
            features['AC'].append(featA['corners'])
            features['AF'].append(featA['fouls'])
                
    assert len(features['ptsH']) == len(features['ptsA']) == len(cuotaNatDB)                    
                
    return features


In [69]:
features = crear_tendencias(n=3)

3 10
3 5

3 8
3 6

3 6
3 6

3 4
3 5

3 5
3 4

3 0
3 4

3 4
3 7

3 5
3 1

3 2
3 5

3 5
3 5

3 3
3 1

3 5
3 12

3 2
3 4

3 5
3 3

3 2
3 4

3 4
3 5

3 2
3 7

3 4
3 10

3 2
3 7

3 3
3 4

3 8
3 3

3 12
3 2

3 6
3 3

3 0
3 4

3 6
3 6

3 2
3 4

3 8
3 3

3 2
3 6

3 1
3 3

3 3
3 8

3 5
3 9

3 3
3 3

3 4
3 0

3 5
3 15

3 2
3 4

3 3
3 2

3 1
3 1

3 6
3 6

3 5
3 4

3 11
3 5

3 10
3 5

3 3
3 3

3 4
3 5

3 3
3 4

3 2
3 2

3 5
3 2

3 6
3 12

3 4
3 3

3 2
3 4

3 3
3 0

3 1
3 0

3 3
3 2

3 4
3 13

3 3
3 8

3 0
3 5

3 8
3 6

3 2
3 1

3 11
3 5

3 8
3 5

3 3
3 1

3 4
3 4

3 1
3 3

3 5
3 1

3 2
3 10

3 5
3 4

3 5
3 1

3 2
3 4

3 3
3 6

3 3
3 1

3 4
3 6

3 0
3 0

3 1
3 4

3 3
3 5

3 3
3 3

3 5
3 3

3 4
3 3

3 6
3 1

3 4
3 3

3 0
3 2

3 1
3 1

3 3
3 5

3 3
3 4

3 5
3 3

3 4
3 4

3 5
3 2

3 6
3 3

3 1
3 6

3 1
3 5

3 3
3 2

3 1
3 4

3 6
3 3

3 4
3 5

3 2
3 4

3 5
3 2

3 6
3 3

3 6
3 2

3 4
3 0

3 3
3 6

3 4
3 3

3 6
3 4

3 0
3 7

3 4
3 2

3 4
3 7

3 5
3 3

3 3
3 5

3 2
3 2

3 3
3 9

3 3
3 2

3 4
3 3

3 3
3 2


3 2
3 2

3 1
3 3

3 2
3 10

3 4
3 8

3 5
3 1

3 1
3 4

3 3
3 2

3 7
3 2

3 0
3 3

3 9
3 3

3 7
3 3

3 3
3 7

3 2
3 1

3 2
3 4

3 6
3 2

3 4
3 3

3 3
3 2

3 4
3 9

3 1
3 1

3 4
3 4

3 1
3 9

3 1
3 2

3 0
3 3

3 2
3 5

3 9
3 8

3 3
3 3

3 2
3 8

3 6
3 6

3 6
3 5

3 5
3 4

3 5
3 8

3 4
3 4

3 3
3 1

3 1
3 4

3 1
3 5

3 7
3 2

3 4
3 6

3 3
3 3

3 2
3 3

3 3
3 1

3 4
3 6

3 7
3 3

3 0
3 2

3 4
3 2

3 6
3 2

3 1
3 9

3 5
3 2

3 7
3 2

3 3
3 2

3 3
3 3

3 4
3 2

3 1
3 3

3 8
3 7

3 3
3 2

3 5
3 5

3 1
3 2

3 4
3 3

3 4
3 8

3 3
3 7

3 4
3 3

3 1
3 5

3 5
3 4

3 1
3 4

3 4
3 1

3 2
3 5

3 2
3 2

3 6
3 3

3 10
3 7

3 6
3 10

3 3
3 5

3 3
3 7

3 8
3 6

3 3
3 2

3 1
3 4

3 0
3 3

3 3
3 1

3 4
3 0

3 3
3 5

3 4
3 5

3 3
3 4

3 7
3 4

3 5
3 3

3 2
3 3

3 3
3 0

3 9
3 1

3 6
3 11

3 3
3 4

3 0
3 3

3 2
3 6

3 2
3 0

3 3
3 5

3 1
3 2

3 3
3 4

3 5
3 2

3 1
3 4

3 6
3 5

3 5
3 4

3 2
3 5

3 4
3 10

3 9
3 1

3 3
3 6

3 5
3 2

3 4
3 3

3 2
3 5

3 4
3 1

3 2
3 4

3 4
3 2

3 3
3 4

3 5
3 7

3 3
3 6

3 4
3

3 3
3 4

3 5
3 3

3 7
3 6

3 6
3 4

3 2
3 10

3 2
3 5

3 6
3 3

3 2
3 8

3 5
3 6

3 4
3 3

3 5
3 3

3 3
3 1

3 6
3 3

3 7
3 6

3 3
3 4

3 2
3 4

3 3
3 2

3 5
3 5

3 3
3 10

3 6
3 8

3 9
3 3

3 2
3 2

3 4
3 7

3 4
3 2

3 2
3 1

3 4
3 2

3 3
3 6

3 10
3 6

3 4
3 6

3 3
3 10

3 9
3 9

3 2
3 8

3 3
3 2

3 5
3 4

3 4
3 3

3 4
3 6

3 2
3 3

3 3
3 1

3 13
3 3

3 1
3 4

3 2
3 2

3 6
3 10

3 5
3 2

3 4
3 4

3 4
3 3

3 8
3 4

3 4
3 1

3 2
3 5

3 6
3 3

3 3
3 10

3 2
3 3

3 5
3 5

3 1
3 5

3 5
3 3

3 2
3 6

3 2
3 4

3 4
3 3

3 5
3 6

3 10
3 5

3 5
3 4

3 7
3 6

3 3
3 6

3 8
3 4

3 3
3 1

3 4
3 5

3 1
3 2

3 4
3 1

3 3
3 4

3 6
3 2

3 6
3 9

3 5
3 7

3 5
3 4

3 2
3 4

3 3
3 2

3 3
3 5

3 3
3 2

3 1
3 8

3 2
3 1

3 5
3 5

3 3
3 3

3 4
3 4

3 3
3 2

3 7
3 9

3 2
3 0

3 4
3 2

3 2
3 5

3 5
3 1

3 1
3 5

3 8
3 3

3 10
3 5

3 4
3 2

3 4
3 4

3 3
3 3

3 1
3 5

3 4
3 1

3 4
3 2

3 2
3 0

3 6
3 4

3 10
3 3

3 2
3 9

3 3
3 3

3 2
3 3

3 2
3 4

3 3
3 6

3 2
3 3

3 2
3 3

3 4
3 2

3 1
3 9

3 11
3 3

3 3
3 5


In [70]:
claves = []

for c in list(features.keys()):
    claves.append('tend_' + str(c))
    
claves

['tend_ptsH',
 'tend_ptsA',
 'tend_goalsH',
 'tend_goalsA',
 'tend_HS',
 'tend_AS',
 'tend_HST',
 'tend_AST',
 'tend_HC',
 'tend_AC',
 'tend_HF',
 'tend_AF']

In [71]:
tend_values = list(features.values())

In [72]:
statsDB_tend3 = statsNatDB_v1

for i in range(0,len(tend_values)):
    statsDB_tend3[claves[i]] = tend_values[i]
    
statsDB_tend3['tend_AC']

0        6.0
1       16.0
2       18.0
3       11.0
4       10.0
        ... 
1821     NaN
1822     NaN
1823     NaN
1824     NaN
1825     NaN
Name: tend_AC, Length: 1826, dtype: float64

Vamos a simplificar el dataset para poder ser usado

In [79]:
res = calcRES(statsDB_tend3)
statsDB_tend3['res'] = res

In [80]:
# LA EXPORTAMOS
# cuotasDB_tend5.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB_tend5.csv',index=False)
# cuotaNatDB.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv',index=False)
# statsDB_tend3.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\statsDB_tend3.csv',index=False)

# LO IMPORTAMOS
cuotaNatDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv')
cuotasDB_tend5 = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB_tend5.csv')
statsDB_tend3 = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\statsDB_tend3.csv')
statsDB_tend3[:5]

,wyId,competitionId,seasonId,roundId,dateutc,winner,teamId_home,score_home,scoreHT_home,teamId_away,...,tend_goalsA,tend_HS,tend_AS,tend_HST,tend_AST,tend_HC,tend_AC,tend_HF,tend_AF,res
0,2576335,524,181248,4406278,2018-05-20 18:45:00,3161,3162,2,2,3161,...,5.0,42.0,20.0,23.0,8.0,15.0,6.0,33.0,35.0,1
1,2576336,524,181248,4406278,2018-05-20 18:45:00,3158,3315,0,0,3158,...,6.0,49.0,46.0,20.0,19.0,20.0,16.0,51.0,36.0,1
2,2576329,524,181248,4406278,2018-05-20 16:00:00,3173,3173,1,0,3172,...,6.0,28.0,33.0,10.0,16.0,12.0,18.0,40.0,27.0,-1
3,2576330,524,181248,4406278,2018-05-20 16:00:00,3165,3165,1,0,3219,...,5.0,41.0,39.0,16.0,14.0,9.0,11.0,39.0,43.0,-1
4,2576331,524,181248,4406278,2018-05-20 16:00:00,3163,3163,1,1,3166,...,4.0,46.0,39.0,18.0,16.0,12.0,10.0,42.0,32.0,-1


In [16]:
cuotasDB_tend5[:-5]

,B365H,B365A,res,tendencia_H,tendencia_A
0,2.40,2.90,1,2.2,1.8
1,4.00,1.85,1,2.4,2.6
2,3.10,2.25,-1,0.8,2.2
3,1.53,5.75,-1,1.4,1.4
4,1.53,6.00,-1,0.8,0.2
...,...,...,...,...,...
1816,2.10,3.70,1,NaN,NaN
1817,3.25,2.30,-1,NaN,NaN
1818,9.50,1.30,1,NaN,NaN
1819,1.17,15.00,-1,NaN,NaN


In [7]:
cuotasDB_tend3 = cuotasDB
tendenciasH, tendenciasA = crear_tendencias(n=3)
cuotasDB_tend3['tendencia_H'] = tendenciasH
cuotasDB_tend3['tendencia_A'] = tendenciasA

cuotasDB_tend3.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotasDB_tend3.csv',index=False)

[3, 3, 3] 3.0
[3, 3, 3] 3.0
[0, 3, 3] 2.0
[0, 1, 3] 1.3333333333333333
[0, 1, 3] 1.3333333333333333
[3, 0, 0] 1.0
[0, 3, 0] 1.0
[0, 1, 3] 1.3333333333333333
[3, 1, 1] 1.6666666666666667
[3, 0, 0] 1.0
[1, 0, 0] 0.3333333333333333
[3, 0, 0] 1.0
[0, 3, 3] 2.0
[3, 1, 3] 2.3333333333333335
[3, 3, 0] 2.0
[1, 1, 0] 0.6666666666666666
[0, 0, 0] 0.0
[0, 0, 1] 0.3333333333333333
[1, 3, 0] 1.3333333333333333
[3, 0, 3] 2.0
[0, 0, 1] 0.3333333333333333
[0, 0, 1] 0.3333333333333333
[0, 1, 0] 0.3333333333333333
[3, 3, 3] 3.0
[1, 0, 1] 0.6666666666666666
[0, 0, 3] 1.0
[1, 3, 0] 1.3333333333333333
[3, 0, 0] 1.0
[0, 0, 0] 0.0
[3, 3, 0] 2.0
[1, 3, 1] 1.6666666666666667
[1, 1, 3] 1.6666666666666667
[0, 0, 1] 0.3333333333333333
[1, 3, 3] 2.3333333333333335
[3, 3, 1] 2.3333333333333335
[3, 3, 3] 3.0
[0, 1, 0] 0.3333333333333333
[3, 0, 3] 2.0
[0, 3, 1] 1.3333333333333333
[0, 1, 0] 0.3333333333333333
[0, 3, 3] 2.0
[3, 1, 0] 1.3333333333333333
[3, 3, 3] 3.0
[0, 0, 1] 0.3333333333333333
[3, 3, 0] 2.0
[3, 1, 0] 

[1, 1, 3] 1.6666666666666667
[3, 1, 1] 1.6666666666666667
[0, 3, 3] 2.0
[0, 1, 3] 1.3333333333333333
[1, 0, 3] 1.3333333333333333
[1, 0, 0] 0.3333333333333333
[1, 3, 0] 1.3333333333333333
[1, 0, 0] 0.3333333333333333
[0, 1, 1] 0.6666666666666666
[3, 0, 3] 2.0
[0, 1, 0] 0.3333333333333333
[0, 1, 0] 0.3333333333333333
[1, 0, 0] 0.3333333333333333
[3, 0, 1] 1.3333333333333333
[0, 3, 1] 1.3333333333333333
[1, 3, 3] 2.3333333333333335
[1, 1, 0] 0.6666666666666666
[3, 3, 1] 2.3333333333333335
[0, 0, 0] 0.0
[1, 1, 0] 0.6666666666666666
[0, 3, 1] 1.3333333333333333
[1, 1, 3] 1.6666666666666667
[0, 3, 0] 1.0
[1, 0, 3] 1.3333333333333333
[1, 3, 3] 2.3333333333333335
[3, 3, 3] 3.0
[1, 1, 0] 0.6666666666666666
[3, 0, 0] 1.0
[0, 0, 3] 1.0
[3, 0, 3] 2.0
[0, 3, 0] 1.0
[0, 0, 0] 0.0
[1, 3, 1] 1.6666666666666667
[0, 0, 0] 0.0
[3, 3, 1] 2.3333333333333335
[3, 1, 0] 1.3333333333333333
[0, 1, 3] 1.3333333333333333
[3, 0, 3] 2.0
[3, 1, 3] 2.3333333333333335
[0, 3, 3] 2.0
[1, 0, 1] 0.6666666666666666
[1, 0,

[3, 0, 1] 1.3333333333333333
[1, 3, 0] 1.3333333333333333
[1, 0, 0] 0.3333333333333333
[0, 0, 0] 0.0
[0, 0, 1] 0.3333333333333333
[3, 1, 3] 2.3333333333333335
[3, 1, 3] 2.3333333333333335
[0, 1, 1] 0.6666666666666666
[1, 1, 3] 1.6666666666666667
[3, 1, 1] 1.6666666666666667
[0, 0, 1] 0.3333333333333333
[3, 3, 3] 3.0
[3, 1, 3] 2.3333333333333335
[1, 3, 1] 1.6666666666666667
[1, 1, 3] 1.6666666666666667
[1, 3, 3] 2.3333333333333335
[0, 0, 1] 0.3333333333333333
[1, 3, 3] 2.3333333333333335
[3, 0, 0] 1.0
[3, 1, 0] 1.3333333333333333
[0, 1, 1] 0.6666666666666666
[0, 1, 1] 0.6666666666666666
[1, 3, 1] 1.6666666666666667
[3, 1, 0] 1.3333333333333333
[0, 3, 0] 1.0
[0, 0, 3] 1.0
[0, 1, 0] 0.3333333333333333
[0, 1, 1] 0.6666666666666666
[1, 1, 0] 0.6666666666666666
[3, 3, 3] 3.0
[3, 1, 0] 1.3333333333333333
[1, 3, 1] 1.6666666666666667
[3, 3, 1] 2.3333333333333335
[3, 0, 1] 1.3333333333333333
[3, 3, 0] 2.0
[1, 0, 3] 1.3333333333333333
[1, 1, 0] 0.6666666666666666
[1, 3, 3] 2.3333333333333335
[0,

[0, 0, 0] 0.0
[1, 0, 0] 0.3333333333333333
[1, 3, 3] 2.3333333333333335
[0, 1, 3] 1.3333333333333333
[1, 0, 3] 1.3333333333333333
[0, 3, 0] 1.0
[3, 3, 0] 2.0
[1, 3, 1] 1.6666666666666667
[1, 3, 1] 1.6666666666666667
[3, 1, 3] 2.3333333333333335
[3, 3, 1] 2.3333333333333335
[1, 1, 3] 1.6666666666666667
[0, 3, 0] 1.0
[3, 3, 0] 2.0
[1, 3, 3] 2.3333333333333335
[3, 0, 0] 1.0
[1, 1, 3] 1.6666666666666667
[3, 3, 0] 2.0
[3, 0, 1] 1.3333333333333333
[3, 0, 1] 1.3333333333333333
[0, 0, 0] 0.0
[3, 1, 0] 1.3333333333333333
[1, 0, 3] 1.3333333333333333
[1, 3, 3] 2.3333333333333335
[0, 3, 3] 2.0
[0, 3, 3] 2.0
[0, 0, 3] 1.0
[0, 0, 0] 0.0
[3, 1, 0] 1.3333333333333333
[1, 3, 3] 2.3333333333333335
[0, 0, 0] 0.0
[3, 3, 3] 3.0
[3, 3, 0] 2.0
[0, 3, 0] 1.0
[1, 0, 3] 1.3333333333333333
[1, 3, 3] 2.3333333333333335
[3, 3, 0] 2.0
[3, 3, 0] 2.0
[0, 0, 1] 0.3333333333333333
[0, 0, 1] 0.3333333333333333
[3, 0, 0] 1.0
[0, 3, 3] 2.0
[1, 3, 3] 2.3333333333333335
[1, 0, 0] 0.3333333333333333
[3, 3, 0] 2.0
[0, 0, 0] 

[3, 1, 1] 1.6666666666666667
[0, 0, 1] 0.3333333333333333
[0, 3, 0] 1.0
[3, 0, 0] 1.0
[1, 0, 0] 0.3333333333333333
[0, 3, 1] 1.3333333333333333
[3, 1, 3] 2.3333333333333335
[3, 3, 0] 2.0
[3, 3, 3] 3.0
[1, 0, 3] 1.3333333333333333
[0, 0, 3] 1.0
[3, 0, 0] 1.0
[1, 3, 0] 1.3333333333333333
[0, 0, 3] 1.0
[0, 0, 3] 1.0
[0, 3, 3] 2.0
[1, 0, 3] 1.3333333333333333
[3, 3, 0] 2.0
[0, 3, 3] 2.0
[1, 1, 0] 0.6666666666666666
[0, 3, 0] 1.0
[3, 3, 1] 2.3333333333333335
[3, 1, 1] 1.6666666666666667
[0, 0, 0] 0.0
[1, 0, 3] 1.3333333333333333
[1, 1, 1] 1.0
[3, 0, 1] 1.3333333333333333
[3, 3, 1] 2.3333333333333335
[3, 0, 3] 2.0
[1, 3, 3] 2.3333333333333335
[0, 1, 1] 0.6666666666666666
[0, 1, 0] 0.3333333333333333
[0, 0, 0] 0.0
[0, 3, 3] 2.0
[0, 3, 3] 2.0
[0, 3, 3] 2.0
[1, 0, 0] 0.3333333333333333
[3, 0, 0] 1.0
[3, 3, 3] 3.0
[0, 3, 0] 1.0
[1, 1, 3] 1.6666666666666667
[0, 3, 3] 2.0
[3, 1, 3] 2.3333333333333335
[3, 3, 0] 2.0
[3, 1, 3] 2.3333333333333335
[1, 1, 0] 0.6666666666666666
[1, 1, 3] 1.66666666666666

[1, 0, 3] 1.3333333333333333
[1, 0, 0] 0.3333333333333333
[1, 0, 3] 1.3333333333333333
[3, 3, 3] 3.0
[3, 0, 3] 2.0
[1, 3, 3] 2.3333333333333335
[0, 1, 0] 0.3333333333333333
[1, 3, 3] 2.3333333333333335
[0, 3, 0] 1.0
[3, 3, 3] 3.0
[1, 0, 0] 0.3333333333333333
[1, 1, 0] 0.6666666666666666
[1, 0, 0] 0.3333333333333333
[1, 0, 0] 0.3333333333333333
[0, 3, 0] 1.0
[0, 1, 0] 0.3333333333333333
[3, 1, 0] 1.3333333333333333
[3, 0, 0] 1.0
[3, 3, 0] 2.0
[0, 3, 0] 1.0
[1, 0, 1] 0.6666666666666666
[0, 0, 1] 0.3333333333333333
[0, 0, 3] 1.0
[1, 0, 0] 0.3333333333333333
[3, 3, 0] 2.0
[3, 0, 3] 2.0
[3, 3, 3] 3.0
[0, 3, 1] 1.3333333333333333
[0, 0, 0] 0.0
[0, 0, 3] 1.0
[3, 3, 3] 3.0
[1, 0, 0] 0.3333333333333333
[3, 3, 3] 3.0
[0, 3, 0] 1.0
[1, 0, 3] 1.3333333333333333
[0, 3, 3] 2.0
[3, 3, 1] 2.3333333333333335
[3, 3, 0] 2.0
[3, 3, 3] 3.0
[3, 0, 0] 1.0
[3, 0, 3] 2.0
[0, 3, 0] 1.0
[0, 0, 0] 0.0
[3, 3, 1] 2.3333333333333335
[3, 0, 1] 1.3333333333333333
[0, 3, 0] 1.0
[0, 3, 3] 2.0
[3, 0, 1] 1.333333333333333

[3, 1, 1] 1.6666666666666667
[3, 0, 1] 1.3333333333333333
[1, 3, 1] 1.6666666666666667
[3, 3, 3] 3.0
[0, 3, 3] 2.0
[0, 3, 0] 1.0
[3, 1, 0] 1.3333333333333333
[1, 1, 0] 0.6666666666666666
[3, 0, 1] 1.3333333333333333
[0, 0, 0] 0.0
[0, 1, 3] 1.3333333333333333
[0, 0, 0] 0.0
[0, 3, 1] 1.3333333333333333
[0, 1, 0] 0.3333333333333333
[1, 1, 0] 0.6666666666666666
[3, 0, 1] 1.3333333333333333
[3, 3, 3] 3.0
[1, 0, 0] 0.3333333333333333
[1, 3, 1] 1.6666666666666667
[0, 0, 3] 1.0
[3, 3, 1] 2.3333333333333335
[0, 3, 0] 1.0
[3, 1, 3] 2.3333333333333335
[3, 1, 1] 1.6666666666666667
[3, 3, 3] 3.0
[0, 0, 3] 1.0
[3, 3, 3] 3.0
[1, 3, 0] 1.3333333333333333
[0, 1, 0] 0.3333333333333333
[3, 1, 0] 1.3333333333333333
[1, 0, 3] 1.3333333333333333
[3, 3, 3] 3.0
[1, 0, 0] 0.3333333333333333
[3, 1, 3] 2.3333333333333335
[3, 0, 1] 1.3333333333333333
[3, 1, 3] 2.3333333333333335
[1, 1, 3] 1.6666666666666667
[3, 3, 0] 2.0
[0, 0, 0] 0.0
[1, 3, 3] 2.3333333333333335
[0, 0, 1] 0.3333333333333333
[3, 3, 3] 3.0
[0, 3, 

[1, 0, 3] 1.3333333333333333
[3, 3, 3] 3.0
[0, 3, 0] 1.0
[1, 3, 3] 2.3333333333333335
[3, 0, 0] 1.0
[0, 3, 3] 2.0
[1, 3, 1] 1.6666666666666667
[3, 3, 3] 3.0
[1, 0, 3] 1.3333333333333333
[0, 0, 0] 0.0
[0, 0, 0] 0.0
[1, 0, 3] 1.3333333333333333
[0, 0, 0] 0.0
[0, 3, 3] 2.0
[0, 0, 3] 1.0
[1, 3, 3] 2.3333333333333335
[1, 3, 0] 1.3333333333333333
[3, 0, 0] 1.0
[3, 3, 0] 2.0
[1, 3, 0] 1.3333333333333333
[0, 0, 3] 1.0
[0, 0, 3] 1.0
[3, 3, 3] 3.0
[3, 0, 0] 1.0
[3, 3, 1] 2.3333333333333335
[3, 3, 0] 2.0
[0, 0, 3] 1.0
[0, 0, 3] 1.0
[0, 3, 0] 1.0
[0, 3, 0] 1.0
[0, 0, 3] 1.0
[0, 3, 0] 1.0
[0, 3, 0] 1.0
[3, 1, 3] 2.3333333333333335
[3, 3, 3] 3.0
[0, 0, 3] 1.0
[3, 0, 0] 1.0
[0, 0, 3] 1.0
[3, 0, 0] 1.0
[3, 0, 0] 1.0
[0, 3, 0] 1.0
[3, 0, 3] 2.0
[3, 3, 0] 2.0
[3, 3, 0] 2.0
[3, 3, 0] 2.0
[3, 1, 3] 2.3333333333333335
[1, 3, 1] 1.6666666666666667
[0, 3, 3] 2.0
[0, 0, 3] 1.0
[3, 0, 1] 1.3333333333333333
[0, 3, 3] 2.0
[0, 0, 0] 0.0
[0, 3, 1] 1.3333333333333333
[3, 0, 0] 1.0
[0, 3, 1] 1.3333333333333333
[3, 0

[0, 3, 1] 1.3333333333333333
[0, 0, 3] 1.0
[3, 0, 0] 1.0
[0, 1, 0] 0.3333333333333333
[3, 0, 0] 1.0
[3, 3, 3] 3.0
[0, 1, 1] 0.6666666666666666
[3, 0, 3] 2.0
[3, 3, 0] 2.0
[0, 1, 1] 0.6666666666666666
[3, 3, 3] 3.0
[3, 1, 1] 1.6666666666666667
[1, 1, 1] 1.0
[0, 0, 0] 0.0
[3, 0, 1] 1.3333333333333333
[3, 0, 3] 2.0
[3, 3, 0] 2.0
[0, 0, 1] 0.3333333333333333
[0, 3, 1] 1.3333333333333333
[1, 0, 3] 1.3333333333333333
[0, 3, 0] 1.0
[1, 1, 0] 0.6666666666666666
[3, 1, 3] 2.3333333333333335
[3, 3, 3] 3.0
[1, 0, 0] 0.3333333333333333
[0, 0, 1] 0.3333333333333333
[0, 3, 3] 2.0
[0, 0, 1] 0.3333333333333333
[1, 3, 0] 1.3333333333333333
[1, 3, 1] 1.6666666666666667
[0, 1, 3] 1.3333333333333333
[0, 1, 0] 0.3333333333333333
[1, 1, 3] 1.6666666666666667
[3, 0, 0] 1.0
[0, 3, 3] 2.0
[1, 1, 3] 1.6666666666666667
[3, 3, 0] 2.0
[3, 1, 0] 1.3333333333333333
[0, 1, 0] 0.3333333333333333
[1, 3, 0] 1.3333333333333333
[0, 1, 3] 1.3333333333333333
[3, 0, 0] 1.0
[0, 0, 3] 1.0
[3, 0, 3] 2.0
[0, 3, 0] 1.0
[3, 3, 3] 

In [10]:
d = cuotasDB_tend3
dif_tend = [y-x for x,y in zip(d['tendencia_H'],d['tendencia_A'])]
assert len(dif_tend) == len(d)

In [12]:
cuotasDB_tend3_norm = cuotasDB_tend3
cuotasDB_tend3_norm['dif_tend'] = dif_tend
cuotasDB_tend3_norm = ft.normalDB(cuotasDB_tend3_norm)
cuotasDB_tend3_norm = cuotasDB_tend3_norm.drop(['tendencia_H', 'tendencia_A'], axis=1)
cuotasDB_tend3_norm.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotasDB_tend3_norm.csv',index=False)

In [ ]:
data = statsDB_tend3
data = data.dropna(axis=0)
data

dif_puntos = data['tend_ptsA'] - data['tend_ptsH']
dif_goles = data['tend_goalsA'] - data['tend_goalsH']
dif_shots = data['tend_AS'] - data['tend_HS']
dif_targets = data['tend_AST'] - data['tend_HST']
dif_corners = data['tend_AC'] - data['tend_HC']

data['dif_puntos'] = dif_puntos
data['dif_goles'] = dif_goles
data['dif_shots'] = dif_shots
data['dif_targets'] = dif_targets
data['dif_corners'] = dif_corners

Lo siguiente...

Intentar visualizar los datos y probar algun modelo con Scikit Learn

Quiero ver si hay alguna correlación entre ambas variables (cuotas, tendencias...), ya que de haberla, no tendria sentido usar ambas variables en el modelo.